# 5. 画像認識による物体検出

<div class="alert alert-block alert-info">
    <b>この章の目的</b>
    <p>画像処理で物体の座標を検出する方法を学習します</p>
</div> 

## セットアップ作業

必要なライブラリをインポートして、初期化を行います。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import rospy
import tf
from utils import *
rospy.init_node("recognition")
rgbd = RGBD()

rvizを起動します．ロボットモデル、カメラ映像、ポイントクラウドが表示されています。

In [ ]:
%%script bash --bg
rviz -d data/5_recognition.rviz > /dev/null 2>&1

## 認識対象の配置

今回、認識対象とするレゴブロックをHSRの前に出現させましょう。

In [ ]:
put_object("e_lego_duplo", 0.6, 0.0, 0.0)

レゴブロックが頭部カメラの画角に入るように頭を少し下げます。rviz上の頭部カメラの映像が変化することを確認しましょう。

In [ ]:
# 頭部を動かします
# 下向きの場合は負、上向きの場合は正の値を引数に与えます
move_head_tilt(-1)

## センサ情報の取得

今回はセンサ情報として、頭部RGB-Dカメラから取得できるポイントクラウド情報を利用します。

以下でRGB-Dカメラから取得したRGB画像を表示することができます。

In [ ]:
# 画像を取得します
image_data = rgbd.get_image()
# 取得した画像を表示してみます
plt.imshow(image_data)

画像データは、480x640x3の3次元配列です。

In [ ]:
image_data.shape

最初の2次元は（y, x）のピクセルに対応します。最後の1次元はRGB形式のピクセル値です。

例えば以下の命令で左上のピクセルのRGB値にアクセスできます。

In [ ]:
image_data[0][0]

ポイントクラウド情報には、各ピクセルの3次元座標値が格納されています。

以下を実行することで、深度情報を表示することができます。

In [ ]:
# ポイントクラウドを取得します
points_data = rgbd.get_points()
# ポイントクラウドの深度情報を表示してみます
plt.imshow(points_data['z'])

深度情報は、480x640の2次元配列です。

In [ ]:
points_data['z'].shape

画像データと同様に、以下の命令で左上のピクセル値にアクセスできます。単位はメートルです。

In [ ]:
points_data['z'][0][0]

## 色空間の変換と色抽出

レゴブロックの色は「水色」なので、色を使ってレゴブロックを検出してみましょう。

「image_data」にはRGB形式で色のデータが格納されています。
RGB形式のデータをそのまま使ってもよいのですが、RGBからHSVに色空間を変換すると照明変化に頑健になることが知られています。

HSVは、色相(Hue)、彩度(Saturation)、明度(Value)によって色を表現します。今回は色相情報のみを利用してレゴブロックの水色を抽出します。

In [ ]:
# 画像の色相の値を取得します
h_image = rgbd.get_h_image()
# 色相画像を表示してみます
plt.imshow(h_image)

色相画像に対して適当な閾値を設定して、レゴブロックのみが抽出されるようにします。

In [ ]:
from ipywidgets import interact

def f(lower = 0, upper = 255):
    yellow_region = (h_image > lower) & (h_image < upper)
    plt.imshow(yellow_region)

interact(f, lower=(0, 255, 5), upper=(0, 255, 5))

閾値を下限130、上限140ぐらいに設定すると安定して抽出できるようです。

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>同様にバナナやリンゴを抽出してみましょう。</p>
    <p>以下を実行することで、バナナとリンゴを出現させることができます。</p>
</div>

In [ ]:
put_object("banana", 0.6, 0.2, 0.0)

In [ ]:
put_object("apple", 1.0, 0.0, 0.0)

## レゴブロックの位置の取得

レゴブロックを抽出できる色相画像の閾値を設定します。

In [ ]:
rgbd.set_h(130, 140)

設定した閾値の色相画像を表示させてみましょう。

In [ ]:
region = rgbd.get_region()
plt.imshow(region)

ポイントクラウド情報から、レゴブロックと考えられる領域のxyz座標の平均値を計算します。

レゴブロックのxyz座標が以下のように計算できました。

In [ ]:
rgbd.get_xyz()

このxyz値は、「head_rgbd_sensor_rgb_frame」基準座標、つまりRGBDカメラ座標上での値です。

## レゴブロックの座標の出力

出力される座標の名前を`lego`にセットしましょう。

In [ ]:
rgbd.set_coordinate_name("lego")

新しいポイントクラウド情報が入力されるたびにbananaの座標情報が出力されます。

「TF」の「Frames」から`lego`を選択してが正しくpublishされているか確認しましょう。

![title](./imgs/5_lego_tf.png)

座標情報はPythonでもアクセスできます。

例えば、絶対座標（map）上での`lego`の位置(x, y)は以下のように取得できます。

In [ ]:
trans = get_relative_coordinate("map", "lego")
x = trans.translation.x
y = trans.translation.y
x, y

`lego`は、絶対座標上で約（x, y）=（0.6, 0.0）の座標に置かれているようです。

## 認識結果を用いた制御

以上の認識器を用いて、HSRにレゴブロックを拾わせてみましょう。

前章の関数`move_wholebody_ik`に認識結果のx, yを引数として与え、逆運動学によってHSRを制御します。

In [ ]:
# ハンドを開く
move_hand(1)
# 逆運動学を使ってHSRの手先を動かします
move_wholebody_ik(x, y, 0.1, 180, 0, 0)
# ハンドを閉じる
move_hand(0)
# アームを初期姿勢に戻す
move_arm_init()

<div class="alert alert-block alert-info">
    <b>課題</b>
    <p>レゴブロックを出現させる位置を変えても上手く拾えるか試してみましょう。</p>
</div>

In [ ]:
# レゴブロックを削除
delete_object("e_lego_duplo")

In [ ]:
# レゴブロックを配置
put_object("e_lego_duplo", 0.6, 0.2, 0.0)

In [ ]:
# 自分で考えてみましょう。この下に入力できます。
